In [277]:
from mip import *

In [278]:
def ler_arquivo(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        linhas = f.readlines()

    #Gerando uma matriz com os valores das linhas
    val_linhas = [linha.strip().split() for linha in linhas]

    #salvando número de variáveis e restrições 
    num_variaveis = int(val_linhas[0][0])
    num_restricoes = int(val_linhas[0][1])

    #salvando um array com os  coeficientes da função objetivo (podem ser floats)
    coef_func_objetivo = list(map(float, val_linhas[1]))

    #salvando uma matriz com os coeficientes das restrições (podem ser floats)
    coef_restricoes = []
    for i in range(num_restricoes):
        restricao = list(map(float, val_linhas[2 + i])) # restrições começam a partir da linha 2 do arquivo
        coef_restricoes.append(restricao)
    
    return num_variaveis, num_restricoes, coef_func_objetivo, coef_restricoes

In [ ]:
def Branch_and_Bound(num_variaveis, num_restricoes, coef_func_objetivo, coef_restricoes):

    zd = float ("inf")
    zp = float ("-inf")
    modelos_solucao_inteira = []

    model = Model(sense=MAXIMIZE, solver_name=CBC)

    #definindo as variaveis
    x = [model.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0, ub=1.0) for i in range(num_variaveis)]

    # função objetivo é a somatorio das variaveis multiplicadas pelos respectivos coeficientes
    model.objective = xsum(coef_func_objetivo[i]*x[i] for i in range(num_variaveis))

    #Adicionando as restrições
    for i in range(num_restricoes):
        model += xsum(coef_restricoes[i][j]*x[j] for j in range(len(coef_restricoes[i])-1)) <= coef_restricoes[i][-1]

    #Rodando o modelo pela primeira vez
    model.optimize()

    # Verificando as variáveis não inteiras
    vars_not_int = []
    for var in model.vars:
        if not (var.x.is_integer()):
            vars_not_int.append(var) #adicionando no array de não inteiras
    
    # caso todas as variaveis sejam inteiras, já encontrou a solução ótima inteira de primeira
    if not(vars_not_int):
        zp = model.objective_value

    zd = model.objective_value
    
    if(zp == zd):
        print("\n-------------------------------------------------------------------------------------------------------------------------------------------\n")
        print("Valor da Solução Ótimas Inteira:", zp)
        print("\nSolução Ótima Inteira:\n")
        for var in modelo.vars:
            print(f"{var.name} = {var.x:.2f}")
        return zp

    #Criando uma lista de modelos
    models = [model]


    #Iniciando o looping de ramificação e podas enquanto houver modelos na lista
    while(models):

        status = models[0].status

        print("\n-------------------------------------------------------------------------------------------------------------------------------------------\n")
        print("Limite Dual (Zd) =", zd)
        print("Limite Primal (Zp) =", zp, "\n")

        print("Valor atual Zi =", models[0].objective_value, "\n")

        #Poda por inviabilidade (não ira ramificar)
        if(status == OptimizationStatus.INFEASIBLE):
            print("Poda por inviabilidade")
            models.pop(0)
            continue

        #Poda por limitante quando o valor da solução relaxada é menor que o zp (não ira ramificar)
        if(models[0].objective_value < zp):
            print("Poda por limitante Zi < zp")
            models.pop(0)
            continue
        
        
        # Verificando as variáveis não inteiras
        vars_not_int = []
        for var in models[0].vars:
            print(f"{var.name} = {var.x:.2f}" + (" (não é inteira)" if not var.x.is_integer() else ""))
            if not var.x.is_integer():
                vars_not_int.append(var) #adicionando no array de não inteiras


        #Se houverem variáveis não inteiras
        if(vars_not_int):
            
            #É preciso encontrar a mais próxima de 0.5
            mais_proximo_05 = min(vars_not_int, key=lambda var: abs(var.x - 0.5))
            print("\nA variável", mais_proximo_05, "será ramificada")


            #Criando duas ramificações a partir da variável não inteira
            model2 = models[0].copy()
            model2 += mais_proximo_05 == 0
            model2.optimize()
            models.append(model2)

            
            model3 = models[0].copy()
            model3 += mais_proximo_05 == 1
            model3.optimize()
            models.append(model3)

        #Poda por integralidade
        else:
            print("\nSolução Inteira!")

            #salva os modelos com solucao inteira no array

            # Atualiza o Zp se for possível, já que a solucao é inteira
            if(zp <= models[0].objective_value):
                zp = models[0].objective_value
                
                #salva os modelos com possivel solução ótima no array
                solucao_inteira = models[0].copy() 
                solucao_inteira.optimize()
                modelos_solucao_inteira.append(solucao_inteira)


        #removendo o modelo atual da lista e calculando o próximo zd entre os modelos que estam na lista
        models.pop(0)
        zd = max((m.objective_value for m in models if m.status != OptimizationStatus.INFEASIBLE), default=zd)
    

    if(modelos_solucao_inteira):
        print("\n-------------------------------------------------------------------------------------------------------------------------------------------\n")
        print("Valor da Solução Ótimas Inteira:", zp)
        for modelo in modelos_solucao_inteira:
            if (modelo.objective_value == zp): # pega os modelos com solução ótima
                print("\nSolução Ótima Inteira:\n")
                for var in modelo.vars:
                    print(f"{var.name} = {var.x:.2f}")
    
    return zp
                

In [280]:
num_variaveis, num_restricoes, coef_func_objetivo, coef_restricoes = ler_arquivo("teste1.txt")

Branch_and_Bound(num_variaveis, num_restricoes, coef_func_objetivo, coef_restricoes)


-------------------------------------------------------------------------------------------------------------------------------------------

Limite Dual (Zd) = 28.749999999999996
Limite Primal (Zp) = -inf 

Valor atual Zi = 28.749999999999996 

x_0 = 0.00
x_1 = 0.87 (não é inteira)
x_2 = 0.00
x_3 = 0.00
x_4 = 1.00
x_5 = 1.00
x_6 = 0.00

A variável x_1 será ramificada

-------------------------------------------------------------------------------------------------------------------------------------------

Limite Dual (Zd) = 28.571428571428573
Limite Primal (Zp) = -inf 

Valor atual Zi = 26.22222222222222 

x_0 = 0.00
x_1 = 0.00
x_2 = 0.78 (não é inteira)
x_3 = 0.00
x_4 = 1.00
x_5 = 1.00
x_6 = 0.00

A variável x_2 será ramificada

-------------------------------------------------------------------------------------------------------------------------------------------

Limite Dual (Zd) = 28.571428571428573
Limite Primal (Zp) = -inf 

Valor atual Zi = 28.571428571428573 

x_0 = 0.00
x_

20.0